In [1]:
# Дано два файла разных форматов. Необходимо проанализировать их структуру и выделить общие хранимые данные. 
# Необходимо создать таблицу для хранения данных в базе данных. 
# Произведите запись данных из файлов разных форматов в одну таблицу. 
# Реализуйте и выполните следующие запросы:
#  вывод первых (VAR+10) отсортированных по произвольному числовому полю строк из таблицы в файл формата json;
#  вывод (сумму, мин, макс, среднее) по произвольному числовому полю;
#  вывод частоты встречаемости для категориального поля;
#  вывод первых (VAR+15) отфильтрованных по произвольному предикату отсортированных по произвольному числовому полю строк 
#  из таблицы в файл формате json.

In [1]:
import pandas as pd
import sqlite3
import pickle
import json

In [3]:
with open('task_3_var_63_part_1.json', 'r', encoding='utf-8') as file:
    data_json = json.load(file)
data_json[0:3]

[{'artist': 'Huey',
  'song': 'Pop, Lock & Drop It - Video Edit',
  'duration_ms': '261026',
  'year': '2006',
  'tempo': '144.09',
  'genre': 'hip hop, pop',
  'explicit': 'False',
  'popularity': '52',
  'danceability': '0.723'},
 {'artist': 'Faith Evans',
  'song': 'I Love You',
  'duration_ms': '267160',
  'year': '2001',
  'tempo': '168.004',
  'genre': 'hip hop, pop, R&B',
  'explicit': 'False',
  'popularity': '53',
  'danceability': '0.489'},
 {'artist': 'Rita Ora',
  'song': 'Anywhere',
  'duration_ms': '215064',
  'year': '2017',
  'tempo': '106.93',
  'genre': 'hip hop, pop, Dance/Electronic',
  'explicit': 'False',
  'popularity': '63',
  'danceability': '0.628'}]

In [4]:
with open('task_3_var_63_part_2.pkl', 'rb') as file:
    data_pkl = pickle.load(file)
data_pkl[0:3]

[{'artist': 'Jennifer Lopez',
  'song': "I'm Real (feat. Ja Rule) - Murder Remix",
  'duration_ms': '262133',
  'year': '2001',
  'tempo': '83.46',
  'genre': 'hip hop, pop, R&B',
  'acousticness': '0.273',
  'energy': '0.587',
  'popularity': '66'},
 {'artist': 'Lady Gaga',
  'song': 'Telephone',
  'duration_ms': '220640',
  'year': '2009',
  'tempo': '122.014',
  'genre': 'pop',
  'acousticness': '0.00521',
  'energy': '0.836',
  'popularity': '69'},
 {'artist': 'Tove Lo',
  'song': 'Talking Body',
  'duration_ms': '238426',
  'year': '2014',
  'tempo': '119.999',
  'genre': 'pop, Dance/Electronic',
  'acousticness': '0.0966',
  'energy': '0.761',
  'popularity': '66'}]

In [5]:
def connect_to_db(file):
    connection = sqlite3.connect(file)
    connection.row_factory = sqlite3.Row
    return connection

In [14]:
# Создание таблицы в базе данных
def create_table():
    conn = sqlite3.connect('database')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS music
                  (id INTEGER PRIMARY KEY AUTOINCREMENT,
                   artist TEXT,
                   song TEXT,
                   duration_ms INTEGER,
                   year INTEGER,
                   tempo FLOAT,
                   genre TEXT,
                popularity INTEGER)''')
    conn.commit()
    conn.close()

In [16]:
# Чтение данных из файлов в таблицу
def insert_data():
    conn = sqlite3.connect('database')
    c = conn.cursor()
    # Чтение данных из файла JSON и запись их в таблицу
    with open('task_3_var_63_part_1.json', 'r', encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        for data in json_data:
            c.execute("INSERT INTO music (artist, song, duration_ms, year, tempo, genre, popularity) VALUES (?, ?, ?, ?, ?, ?, ?)",
                      (data['artist'], data['song'], data['duration_ms'], data['year'], data['tempo'], data['genre'], data['popularity']))        
            
    # Чтение данных из файла PKL и запись их в таблицу
    with open('task_3_var_63_part_2.pkl', 'rb') as pickle_file:
        pickle_reader = pickle.load(pickle_file)
        for item in pickle_reader:
            c.execute("INSERT INTO music (artist, song, duration_ms, year, tempo, genre, popularity) VALUES (?, ?, ?, ?, ?, ?, ?)",
                      (item['artist'], item['song'], item['duration_ms'], item['year'], item['tempo'], item['genre'], data['popularity']))
    
    conn.commit()
    conn.close()

In [17]:
# Вывод первых 73 отсортированных по числовому полю ['popularity'] в файл json
def sort_by_popularity():
    conn = sqlite3.connect('database')
    c = conn.cursor()
    c.execute("SELECT * FROM music ORDER BY popularity LIMIT 73")
    result = c.fetchall()
    with open('sorted_properties_63_3.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()  

In [18]:
# Вывод суммы, минимума, максимума и среднего значения по полю ['tempo']
def floors_statistics():
    conn = sqlite3.connect('database')
    c = conn.cursor()

    c.execute("SELECT SUM(tempo), MIN(tempo), MAX(tempo), AVG(tempo) FROM music")
    result = c.fetchone()

    conn.close()

    return result

In [19]:
# Вывод частоты встречаемости для категориального поля ['song']
def song_frequency():
    conn = sqlite3.connect('database')
    c = conn.cursor()

    c.execute("SELECT song, COUNT(*) FROM music GROUP BY song")
    result = c.fetchall()

    conn.close()

    return result

In [22]:
# Вывод первых 78 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['year'] в файл json
def filter_and_sort_by_year():
    conn = sqlite3.connect('database')
    c = conn.cursor()

    c.execute("SELECT * FROM music WHERE year > 2010 ORDER BY year LIMIT 78")
    result = c.fetchall()

    with open('filtered_properties_63_3.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()  

In [23]:
connect_to_db('database.db')
create_table()
insert_data()
sort_by_popularity()
floors_statistics()
song_frequency()
filter_and_sort_by_year()  